In [ ]:
import pickle
best_params = pickle.load(open("../pickle_files/baseline.pickle", "rb"))
best_params

In [ ]:
!pip install pandas numpy nltk scikit-learn wordcloud seaborn gensim tensorflow imblearn xgboost matplotlib unrar pyunpack more-itertools patool keras-tqdm > /dev/null

In [1]:
import pandas as pd
import sklearn
import numpy as np
from imblearn.over_sampling import SMOTE,RandomOverSampler
from sklearn import metrics
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from gensim.test.utils import common_texts
from gensim.models import Word2Vec
import nltk

c:\Users\user\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\user\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
c:\Users\user\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [3]:
from nlp_ai_utils import *

In [4]:
LARGER_DATASET_PATH = '../larger_dataset/'
PREPROCESSED_CHUNKS_PATH = LARGER_DATASET_PATH + 'preprocessed_data_chunks/'
if not os.path.exists(LARGER_DATASET_PATH):
    os.mkdir(LARGER_DATASET_PATH)

if not os.path.exists(PREPROCESSED_CHUNKS_PATH):
    os.mkdir(PREPROCESSED_CHUNKS_PATH)

#get all names of downloaded files

all_file_names = get_all_file_names('chunk_',60)
#read all chunks into a list
list_dfs = read_chunks(all_file_names,PREPROCESSED_CHUNKS_PATH,'.csv')
#concatenate all chunks into a singular df
df = group_up_chunks(list_dfs)
#check how much of the data was actually downloaded
percent_loaded = check_no_missing_data(df.shape[0],6990280)
percent_loaded

100.0

In [6]:
df['stars'].value_counts()

5.0    3231627
4.0    1452918
1.0    1069561
3.0     691934
2.0     544240
Name: stars, dtype: int64

In [ ]:
df1 = pd.read_csv("../larger_dataset/data_chunks/chunk_1.csv")
df2 = pd.read_csv("../larger_dataset/preprocessed_data_chunks/chunk_1.csv")

In [ ]:
test = [len(text.split()) for text in df1['text']]

In [ ]:
test

In [ ]:
test

In [ ]:
text = df1['text'][0]

In [ ]:
text

In [ ]:
remove_stop_words(text)

In [ ]:
process_corpus(remove_stop_words(text))

In [ ]:
remove_stop_words(process_corpus(text))

In [ ]:
process_corpus(remove_stop_words(text))

In [ ]:
df2['text'][0]

In [ ]:
test_list = ['love','hate','peace','pizza']

In [ ]:
sid = SentimentIntensityAnalyzer()
sent_polarity_words = [sid.polarity_scores(item) for item in test_list]

In [ ]:
sent_polarity_words[3]

In [ ]:
df = pd.read_csv("../smaller_dataset/yelp coffee/raw_yelp_review_data_processed.csv")
df_unprocessed = pd.read_csv("../smaller_dataset/yelp coffee/raw_yelp_review_data.csv")

In [ ]:
X = df['full_review_text']

y = df['star_rating']

In [ ]:
y

In [ ]:
def encode_sent(sentiments,positive_label = 1,negative_label = 2,neutral_label = 3):
    """
    This function takes in all translated labels (labels transformed for star ratings to sentiments), and converts them to numerical values.

    positive_label: x => Positive Integer, Default = 1

    negative_label: x => Positive Integer, Default = 2

    neutral_label: x => Positive Integer, Default = 3

    """
    translated_labels = sentiments
    encoded_sent = []
    for label in translated_labels:
        if label == 'Positive Sentiment':
            encoded_sent.append(positive_label)
        elif label == 'Negative Sentiment':
            encoded_sent.append(negative_label)
        else:
            encoded_sent.append(neutral_label)
    return encoded_sent

In [ ]:
translated_labels = translate_labels(y)

In [ ]:
encoded_labels = encode_sent(translated_labels,3,1,2)

In [ ]:
encoded_labels

In [ ]:
translated_labels = translate_labels(y)
y = pd.Series(encode_sent(translated_labels))

In [ ]:
y.unique()

In [ ]:
samples = get_classes_count(y,start_label = 1)

In [ ]:
class_weights = compute_class_weight('balanced', classes=list(y.unique()), y=y)

In [ ]:
LABELS_COUNT = len(y.unique())

In [ ]:
LABELS_COUNT

In [ ]:
visualize_ratings_pie(y,use_dict = samples)

In [ ]:
test1 = range(1,6)
test2 = [1,2,3,4,5]

print(test1 == test2)

print(test1)

In [ ]:
def tokenize_review(text):
    return nltk.word_tokenize(text)

In [ ]:
reviews_tokens = [tokenize_review(review) for review in X]

In [ ]:
model = Word2Vec(sentences=reviews_tokens, vector_size=100, window=5, min_count=1, workers=4)
# model.save("test_w2v.model")

In [ ]:
X = np.zeros((len(reviews_tokens), model.vector_size))
labels = y
for i, review in enumerate(reviews_tokens):
    for word in review:
        if word in model.wv:
            X[i] += model.wv[word]
    X[i] /= len(reviews_tokens)

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(X,labels,stratify=labels,test_size=0.2,random_state=42)

In [ ]:
# smote = SMOTE(random_state = 42)
# x_train,y_train = smote.fit_resample(x_train,y_train)

In [ ]:
clf = RandomForestClassifier(random_state=42)
clf.fit(x_train, y_train)
y_pred = clf.predict(x_test)
accuracy = metrics.accuracy_score(y_test,y_pred)

In [ ]:
accuracy

In [ ]:
test_review = "I like this place, but it is not my favourite at all."
test_token = nltk.word_tokenize(test_review)
test_vector = np.zeros(model.vector_size)
for word in test_token:
    if word in model.wv:
        test_vector += model.wv[word]
test_vector /= len(test_token)
predicted_label = clf.predict([test_vector])[0]

In [ ]:
predicted_label